# Multiple Output Formats

## Interface Definition for Scripts

Since GRASS tools are executable scripts (or generally programs), the interface of a GRASS tool is the command line interface of a Python script. A dedicated function _grass.script.parser_ takes care of processing the command line arguments based on the interface description specified in a Python comment with a key-value syntax defined by GRASS GIS.

The following is an example of a script which takes two parameters: name of a vector map and name of a raster map:

In [ ]:
%%writefile viewscape.py
#!/usr/bin/env python

# %module
# % description: Compute viewshed and compute statistics about visible parts of sample layers
# % keyword: raster
# % keyword: statistics
# % keyword: viewshed
# %end
# %option G_OPT_R_ELEV
# % description: Name of input elevation raster map
# %end
# %option G_OPT_M_COORDS
# %end
# %option G_OPT_R_INPUTS
# %end
# %option G_OPT_F_OUTPUT
# %end


import atexit
import subprocess
import sys
import csv
import json
import io

import grass.script as gs


def output_results(results, sample_rasters, file_name, file_format):
    if file_format == "json":
        with open(file_name, "w", encoding="utf-8") as json_file:
            json.dump(results, json_file, ensure_ascii=False, indent=4)
    elif file_format == "csv":
        with open(file_name, "w", newline="", encoding="utf-8") as csv_file:
            header = ["name"]
            header.extend(results[sample_rasters[0]].keys())
            writer = csv.DictWriter(csv_file, fieldnames=header)
            writer.writeheader()
            for key, value in results.items():
                row = {"name": key}
                row.update(value)
                writer.writerow(row)
    else:
        raise ValueError(f"Unsupported or invalid format: {file_format}")


def clean(name):
    gs.run_command("g.remove", type="raster", name=name, flags="f", superquiet=True)


def viewshed(
    elevation,
    coordinates,
    sample_rasters,
    output,
    file_format,
):
    viewshed = gs.append_node_pid("tmp_viewshed")
    atexit.register(clean, viewshed)
    gs.run_command(
        "r.viewshed",
        input=elevation,
        output=viewshed,
        coordinates=coordinates,
        flags="cb",
    )
    gs.run_command("r.null", map=viewshed, setnull=0)
    results = {}
    for name in sample_rasters:
        table_data = gs.read_command(
            "r.univar",
            map=name,
            zones=viewshed,
            quiet=True,
            flags="t",
            separator="comma",
        )
        # While we could use .strip().splitlines()[-1].split(",") here,
        # using a proper CSV reader is more robust.
        reader = csv.DictReader(io.StringIO(table_data))
        for row in reader:
            del row["zone"]
            del row["label"]
            del row["non_null_cells"]
            del row["null_cells"]
            results[name] = row
    output_results(
        results=results,
        sample_rasters=sample_rasters,
        file_name=output,
        file_format=file_format,
    )


def main():
    options, flags = gs.parser()
    coordinates = options["coordinates"].split(",")
    sample_rasters = options["input"].split(",")
    viewshed(
        elevation=options["elevation"],
        coordinates=(float(coordinates[0]), float(coordinates[1])),
        sample_rasters=sample_rasters,
        output=options["output"],
        file_format="json",
    )


if __name__ == "__main__":
    main()

As before, we will make the script executable:

In [ ]:
!chmod u+x viewscape.py

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec ./viewscape.py elevation=elevation coordinates=637_657,222_899 input=elevation,ndvi output="data.txt" --o
!cat data.txt

## Task: Add Parameter for Output Format

### Explore Interface of an Existing Tool

In [ ]:
!grass --tmp-location XY --exec v.db.select --script

## More Than on Input

In [ ]:
!grass ~/grassdata/nc_spm_08_grass7/foss4g --exec i.vi red=lsat7_2002_30 nir=lsat7_2002_40 viname=ndvi output=ndvi

## Task: Divide Parameters into Sections

```python
# % guisection: Output
```